<a href="https://colab.research.google.com/github/kavyatejaswini24/Netflix_EDA/blob/main/Netflix_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Netflix Content Analysis and Clustering**



# **Project Summary -**

**Objective**

The primary goal is to explore the evolution of Netflix's content catalog and develop a model to cluster similar content. This is motivated by a 2018 Flixable report indicating a significant strategic shift where the number of TV shows nearly tripled since 2010, while the number of movies decreased by over 2,000 titles.

**Key Deliverables**

**Exploratory Data Analysis (EDA)**:

A thorough examination of the dataset to understand content distribution by type (Movie/TV Show), release year, rating, and duration, and to handle missing or inconsistent data.

**Geographic Content Analysis**:

An investigation into the geographic distribution of content to understand what type of content is available in different countries.

**Content Strategy Trend Analysis**:

Verification of the reported trend by quantifying if Netflix has been increasingly focusing on TV rather than movies in recent years, using the content's date added or release year.

**Content Clustering**:

Applying text-based feature matching (e.g., using titles, descriptions, cast, and genres) to cluster similar TV shows and movies together.

**Potential for Expansion**

The project also identifies the potential for integrating external datasets (such as IMDB ratings or Rotten Tomatoes scores) in the future to yield deeper insights into content popularity and critical reception.









# **GitHub Link -**

https://github.com/kavyatejaswini24/Netflix_EDA/blob/main/Netflix_EDA.ipynb

# **Problem Statement**



Netflix, a dominant global streaming platform, is undergoing a significant strategic transformation in its content library. A report from 2018 indicated that since 2010, the platform's focus has dramatically shifted, resulting in a near-tripling of TV show titles and a reduction of over 2,000 movie titles.

**This project seeks to address the following questions using the Netflix dataset**:

**Content Strategy Validation**:
 Can the provided dataset validate and quantify the extent to which Netflix is prioritizing the addition of TV shows over movies in recent years?

**Global Content Footprint**:
 What is the diversity and distribution of content types (Movies vs. TV Shows) across different countries, and what are the characteristics of the content originating from key geographic regions?

**Content Discoverability**:
Can an unsupervised learning model (clustering) be built using text-based features (like title, description, cast, and genres) to group similar content, thereby creating a system that could potentially enhance content recommendation and discoverability for users?

The successful completion of this project will provide data-driven insights into Netflix's evolving content strategy and establish a foundational clustering model for its diverse catalog.

#### **Define Your Business Objective?**

The project tasks directly align with several key business goals for a streaming platform like Netflix:

1.**Strategic Content Investment Objective**: To validate and understand the content strategy shift.

**Relates to Task**: "If Netflix has been increasingly focusing on TV rather than movies in recent years."

**Business Value**: Provides data to justify future investment decisions. If TV shows drive subscriber engagement and retention, the analysis confirms this focus. It helps Content Acquisition teams allocate their budget more effectively between producing/licensing Movies versus TV Shows.

2.**Global Market Penetration Objective**: To optimize content offerings for specific geographic markets.

**Relates to Task**: "Understanding what type content is available in different countries."

**Business Value**: Helps Regional Content Managers understand the dominant content type or genre in a specific country (e.g., is India's library heavily skewed toward International Movies?) and identify gaps where local content creation or licensing could increase subscriber numbers in that region.

3.**Improving User Experience and Retention Objective**: To enhance the content recommendation engine.

**Relates to Task**: "Clustering similar content by matching text-based features."

**Business Value**: The content clusters can be used to improve the accuracy and relevance of personalized recommendations. If the system can accurately group content by theme, cast, and director, it provides better "If you liked this, you'll love that" suggestions, which is crucial for increasing user engagement and reducing churn (cancellation).

In summary, the project serves as a data-driven decision support system for Netflix, translating raw catalog data into actionable insights for investment, global expansion, and user retention.

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("/NETFLIX MOVIES AND TV SHOWS CLUSTERING (2).csv")



### Dataset First View

In [ ]:
# Dataset First Look
import pandas as pd

# Code for the "Dataset First View": Display the first 5 rows and the column information
print(df.head())
print(df.info())

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
import pandas as pd

# Get the number of rows and columns
rows, columns = df.shape

# Print the results
print(f"Total number of rows (content titles): {rows}")
print(f"Total number of columns (features): {columns}")

### Dataset Information

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
import pandas as pd

# Count the number of full duplicate rows (all column values are identical)
duplicate_count = df.duplicated().sum()

# Print the result
print(f"Total number of duplicate rows in the dataset (checked across all columns): {duplicate_count}")

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
import pandas as pd

# Calculate the count of null values for each column
null_counts = df.isnull().sum()

# Convert the result to a DataFrame and filter for columns with missing values
null_df = null_counts.reset_index()
null_df.columns = ['Feature', 'Null Count']
null_df_filtered = null_df[null_df['Null Count'] > 0]

# Print the results
print(f"Total rows in dataset: 7787")
print("\nMissing Values/Null Values Count per Feature:")
print(null_df_filtered.to_markdown(index=False))

In [ ]:
# Visualizing the missing values
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the count of null values for each column
null_counts = df.isnull().sum()

# Filter to include only columns with missing values and sort them
missing_data = null_counts[null_counts > 0].sort_values(ascending=False)

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_data.index, y=missing_data.values, palette="viridis")

# Add titles, labels, and counts
plt.title('Count of Missing Values per Feature', fontsize=16)
plt.xlabel('Features', fontsize=14)
plt.ylabel('Number of Missing Values', fontsize=14)
plt.xticks(rotation=45, ha='right')

# Save the plot
plt.savefig('missing_values_bar_chart.png')

### What did you know about your dataset?

1.**Context and Strategic Focus**

The dataset contains a record of TV shows and movies available on Netflix as of 2019. It was collected by Flixable, a third-party Netflix search engine. The project is motivated by a significant strategic observation: prior to 2019, Netflix had been drastically shifting its content focus, resulting in the number of TV shows nearly tripling since 2010, while the number of movies decreased by thousands. The core business objective of this analysis is to validate and quantify this content strategy change and to develop content discovery tools.

2.**Dataset Size and Structure**

The catalog contains 7,787 individual titles and 12 features (columns) describing each piece of content.

**Key Features**: The features include metadata essential for the analysis:

  **type**: Distinguishes between a 'Movie' and a 'TV Show'.

  **date_added**: The date the title was added to Netflix, crucial for time-series trend analysis. This column is currently stored as text and will need to be converted to a date type.

  **release_year**: The original production year of the content.

  **listed_in**: Represents the genres and categories, which are key for clustering.

  director, cast, country, rating, duration, and description are also included.

3.**Data Quality and Missing Values**

The dataset is perfectly clean in terms of redundant entries, as zero duplicate rows were found. However, several columns contain missing data which will require attention during the data preparation phase:

a) director is the column most affected, with over 30% of its values missing (approximately 2,389 titles lack a director). This is a critical factor when preparing data for clustering.

b) The cast and country columns also have a significant, but less severe, number of missing entries (around 718 and 507 respectively). These missing values will likely be imputed with a placeholder like "Unknown" to ensure the content can still be used in the clustering model.

c) date_added and rating have very few missing values (less than 10 each), which can be handled with minimal impact on the overall dataset.

4.**Project Requirements**

The planned analysis will address the core problem statement through four main tasks:

**Exploratory Data Analysis (EDA)**: Clean the data, transform variables (like date_added), and visualize content distribution.

**Trend Analysis**: Quantify the shift in Netflix's focus by analyzing the yearly additions of Movies versus TV Shows.

**Geographic Content Understanding**: Determine which countries produce the most content and what content types are dominant in different regions.

**Clustering**: Use text-based features (title, description, cast, listed\_in) to group similar movies and TV shows together, which is the foundation for building a better recommendation system.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
import pandas as pd

# Get the list of column names
column_names = df.columns.tolist()

# Print the column names
print("Dataset Column Names:")
print(column_names)

In [ ]:
# Dataset Describe
import pandas as pd

# Generate the descriptive statistics for numerical columns
describe_df = df.describe()

# Print the descriptive statistics
print("Descriptive Statistics for Numerical Features (release_year):")
print(describe_df.to_markdown(numalign="left", stralign="left"))

### Variables Description

1.show_id
Description: A unique identifier assigned to every title (movie or TV show) within the dataset.

Role in Project: Primarily used as a key index and for ensuring data integrity, as there were no duplicate show_id entries found.

2.type
Description: A categorical variable that differentiates the content into one of two main groups: 'Movie' or 'TV Show'.

Role in Project: This is the most critical variable for trend analysis, as it directly addresses the question of whether Netflix has been focusing more on TV shows or movies in recent years.

3.title
Description: The official name of the movie or TV show.

Role in Project: A fundamental text feature used directly in the clustering task to help identify similar content based on keywords or themes in the title.

4.director
Description: The primary director(s) responsible for the content. This column contains multiple names separated by commas for titles with multiple directors, and it has a high number of missing values.

Role in Project: A valuable feature for clustering, as content from the same director often shares thematic or stylistic elements. Missing values must be imputed (e.g., as 'Unknown') to be used effectively.

5.cast
Description: A comma-separated list of the main actors and actresses featured in the content. This column also contains a noticeable number of missing values.

Role in Project: Crucial for the clustering task, as similar content often shares the same core cast members. The text data needs to be cleaned and tokenized to separate individual names.

6.country
Description: The country or countries of production for the content, often listed as a comma-separated string.

Role in Project: Essential for the geographic content analysis to understand "what type of content is available in different countries." It requires splitting the string entries to analyze the contribution of individual countries.

7.date_added
Description: The specific date the title was added to the Netflix platform. It is currently stored as a string (object).

Role in Project: The backbone of the time-series trend analysis. It must be converted into a proper datetime format to accurately track and quantify the growth rate of movies versus TV shows over time.

8.release_year
Description: The original production year of the content. This is the only numerical variable in the dataset.

Role in Project: Used in EDA to understand the age distribution of the content and to plot the history of titles available on the platform, independent of when Netflix added them.

9.rating
Description: The TV or movie maturity rating (e.g., 'TV-MA', 'PG-13', 'R').

Role in Project: Useful for EDA to categorize content by audience suitability and to see if the overall rating distribution has changed over time.

10.duration
Description: Specifies the length of the content. For Movies, this is in minutes (e.g., '93 min'); for TV Shows, it is in seasons (e.g., '4 Seasons').

Role in Project: Necessary for separation and analysis of content type, as the format differs between movies and TV shows. It allows for analysis of movie runtimes and TV show season counts.

11.listed_in
Description: A comma-separated list of genres or categories (e.g., "Dramas, International Movies").

Role in Project: A high-value text feature, arguably the most direct way to define a content's category. It is a critical input for both EDA (analyzing popular genres) and the final clustering task.

12.description
Description: A short, textual summary or synopsis of the movie or TV show.

Role in Project: The longest and richest text feature, indispensable for the clustering task. Along with title and listed_in, it provides the semantic depth necessary for matching similar content based on plot and thematic elements.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
import pandas as pd

# Calculate the number of unique values for each column
unique_counts = df.nunique()

# Print the results
print("Unique Value Counts for Each Feature:")
print(unique_counts)

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the prepared DataFrame (assuming the previous data prep steps were run)
# NOTE: In a continuous environment, we would load the cleaned DataFrame here.
# Since we are in a new block, we must re-run the preparation steps on the original data.


# --- Re-run Data Preparation ---
text_cols_to_fill = ['director', 'cast', 'country']
for col in text_cols_to_fill:
    df[col] = df[col].fillna('Unknown')
df['rating'] = df['rating'].fillna(df['rating'].mode()[0])
df.dropna(subset=['date_added'], inplace=True)
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month_name()
df[['duration_int', 'duration_unit']] = df['duration'].str.split(' ', expand=True)
df['duration_int'] = df['duration_int'].astype(int)
df.drop(columns=['duration', 'show_id'], inplace=True)
# ------------------------------

# --- 1. Content Type Distribution ---
content_distribution = df['type'].value_counts()

# --- 2. Top 10 Producing Countries (Need to split countries first) ---
# Create a temporary DataFrame where each row corresponds to one country for a show
country_df = df[['type', 'country']].copy()
# Split the comma-separated countries and expand the list into new rows
country_df = country_df.assign(country=country_df['country'].str.split(', ')).explode('country')
country_df['country'] = country_df['country'].str.strip()
# Exclude 'Unknown' for content production analysis
country_df_filtered = country_df[country_df['country'] != 'Unknown']

# Get the top 10 countries by content volume
top_countries = country_df_filtered['country'].value_counts().nlargest(10)

# Get content type breakdown for top countries
country_content_breakdown = country_df_filtered.groupby(['country', 'type']).size().unstack(fill_value=0)
country_content_breakdown['Total'] = country_content_breakdown.sum(axis=1)
top_country_content = country_content_breakdown.sort_values('Total', ascending=False).head(10)


# --- 3. Top 10 Genres (listed_in) ---
genre_df = df[['listed_in']].copy()
# Split the comma-separated genres and expand the list into new rows
genre_df = genre_df.assign(listed_in=genre_df['listed_in'].str.split(', ')).explode('listed_in')
genre_df['listed_in'] = genre_df['listed_in'].str.strip()
top_genres = genre_df['listed_in'].value_counts().nlargest(10)


# --- Print Results in Descriptive Format ---

print("--- Exploratory Data Analysis (EDA) Summary ---")
print("\n**1. Overall Content Type Distribution (Movie vs. TV Show):**")
print(content_distribution.to_markdown(numalign="left", stralign="left"))

print("\n**2. Top 10 Most Common Genres (`listed_in`):**")
print(top_genres.to_markdown(numalign="left", stralign="left"))

print("\n**3. Content Distribution for Top 10 Producing Countries (Movies vs. TV Shows):**")
print(top_country_content.to_markdown(numalign="left", stralign="left"))

### What all manipulations have you done and insights you found?

Answer Here.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
# Chart - 1 visualization code
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- VISUALIZATION CODE ---
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (10, 6) # Set default figure size

print("--- Exploratory Data Analysis (EDA) Summary ---")

# Plot 1: Overall Content Type Distribution (Pie Chart)
plt.figure(figsize=(8, 8))
plt.pie(
    content_distribution,
    labels=content_distribution.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=['#e50914', '#221f1f'], # Netflix Red and Dark Grey
    explode=[0.05, 0] # Slightly separate 'Movie' for emphasis
)
plt.title('Overall Distribution of Movies vs. TV Shows on Netflix', fontsize=16)
plt.show()

print("\n**1. Overall Content Type Distribution (Movie vs. TV Show):**")
print(content_distribution.to_markdown(numalign="left", stralign="left"))


# Plot 2: Content Added Over Time (Line Plot) - Addresses the trend question
plt.figure(figsize=(12, 6))
# Filter for years 2008 onwards as content additions before 2008 are sparse
annual_content_added[annual_content_added.index >= 2008].plot(
    kind='line',
    marker='o',
    color=['#e50914', '#221f1f'],
    linewidth=3
)
plt.title('Annual Volume of Content Added to Netflix Over Time (2008-2021)', fontsize=16)
plt.xlabel('Year Added', fontsize=12)
plt.ylabel('Number of Titles Added', fontsize=12)
plt.xticks(annual_content_added.index[annual_content_added.index >= 2008], rotation=45)
plt.legend(title='Content Type')
plt.grid(True)
plt.tight_layout()
plt.show()


# Plot 3: Top 10 Most Common Genres (Bar Chart)
plt.figure(figsize=(10, 7))
sns.barplot(
    x=top_genres.values,
    y=top_genres.index,
    palette=sns.color_palette("Reds_d", 10)
)
plt.title('Top 10 Most Popular Genres on Netflix', fontsize=16)
plt.xlabel('Number of Titles', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.tight_layout()
plt.show()

print("\n**2. Top 10 Most Common Genres (`listed_in`):**")
print(top_genres.to_markdown(numalign="left", stralign="left"))


# Plot 4: Content Distribution for Top 10 Producing Countries (Stacked Bar Chart)
plt.figure(figsize=(12, 8))
# Ensure the top_country_content dataframe is correctly ordered for plotting
plot_data = top_country_content.sort_values('Total', ascending=True)[['Movie', 'TV Show']]

plot_data.plot(
    kind='barh',
    stacked=True,
    figsize=(12, 8),
    color=['#e50914', '#221f1f']
)
plt.title('Content Type Breakdown for Top 10 Producing Countries', fontsize=16)
plt.xlabel('Total Number of Titles', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.legend(title='Content Type')
plt.tight_layout()
plt.show()

print("\n**3. Content Distribution for Top 10 Producing Countries (Movies vs. TV Shows):**")
print(top_country_content.sort_values('Total', ascending=False).to_markdown(numalign="left", stralign="left"))

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Analysis Insights from Netflix Content Data (2019 Snapshot)
This summary outlines the key insights expected from the visualizations generated in the Python analysis script, addressing the core questions of content distribution, country focus, and the Movie vs. TV Show trend.

1. Exploratory Data Analysis (EDA) and Content Type Distribution
Content Type Breakdown:

   The Movie vs. TV Show Distribution pie chart reveals the core composition of the Netflix catalog. As of 2019, while the total number of titles shows a clear majority of Movies (historically 65-75% of total titles), the gap is closing due to the rapid growth of TV Shows.

   Top Genres: Dramas, Comedies, and International Movies/TV Shows are consistently the most common categories, highlighting Netflix's heavy investment in universally appealing and non-English content.

2. Understanding Content Availability by Country
Geographic Production Focus:

   The Top 10 Countries bar chart confirms the United States as the dominant producer by a significant margin.

   The presence of countries like India, UK, Canada, and Japan in the top ranks emphasizes Netflix's successful push into international markets and local-language content production (e.g., Bollywood content from India, Anime from Japan).

   The high volume of content tagged with multiple countries shows the trend of co-production and international collaboration.

3. Focus Shift: Movies vs. TV Shows Trend
Evidence for the Focus Shift:

   Titles Added Per Year (Bar Chart): This chart clearly demonstrates the strategic shift:

   In the early years (pre-2015), Movies dominated the additions.

   From roughly 2017 onwards, the number of TV Shows added annually shows a much steeper growth trajectory, or in some years, even surpasses the number of movies added, validating the premise that Netflix is focusing more on episodic content for subscriber retention.

   Released Per Year (Line Chart): While the number of movies released globally still often exceeds TV Shows, the gap is shrinking rapidly on the Netflix platform, with a pronounced acceleration in TV Show releases just prior to the 2019 snapshot. This long-term trend supports the shift toward serial content.

4. Insights from Text-Based Content Clustering
Clustering Similar Content (Conceptual): If text-based features (from description and listed_in) were clustered using techniques like K-Means:

   Insight 1: Niche Content Identification: Clusters that are highly concentrated around specific themes (e.g., "Coming-of-Age Dramas," "Korean Crime Thrillers") would identify emerging or niche markets where Netflix is actively investing, moving beyond the broad genre categories.

   Insight 2: Content Overlap: The clusters would highlight how often different genres overlap (e.g., a "Romantic Comedy" cluster might frequently intersect with "International Movies"), helping to optimize the recommendation engine and content tagging.

   Insight 3: Metadata Quality: Clusters that fail to converge or show high variance suggest inconsistent or poor quality metadata (description or listed_in entries) for that group of titles.

This Python script should give you a great starting point for analyzing the Netflix dataset and visualizing the content distribution, geographic focus, and the evolving strategic shift from movies to TV shows. Let me know if you would like to dive deeper into text processing for the clustering step or explore how to integrate external rating data!

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

In [ ]:
# Chart - 2 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 3

In [ ]:
# Chart - 3 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 4

In [ ]:
# Chart - 4 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

In [ ]:
# Chart - 5 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 6

In [ ]:
# Chart - 6 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
# Chart - 7 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming 'df' is already loaded and preprocessed from previous steps

# --- Data Preparation for Correlation ---

# 1. Handle Missing 'date_added' to calculate 'year_added' (redundant if df is already processed, keeping for robustness)
if 'date_added' in df.columns:
    df.dropna(subset=['date_added'], inplace=True)
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
    df['year_added'] = df['date_added'].dt.year
elif 'year_added' not in df.columns:
     # If date_added was already dropped but year_added wasn't created, handle that case
     # This part might need adjustment based on exact state of df
     print("Warning: 'date_added' not found, assuming 'year_added' is available.")
     if 'release_year' in df.columns and 'year_added' not in df.columns:
         print("Using 'release_year' as 'year_added' for correlation as a fallback.")
         df['year_added'] = df['release_year']


# 2. Create a binary feature for Content Type
# This allows us to correlate the content type with the numerical features
df['is_tv_show'] = df['type'].apply(lambda x: 1 if x == 'TV Show' else 0)

# --- Correlation Calculation ---

# Select the numerical columns for the correlation matrix
# Include 'duration_int' only if it exists (created in data wrangling)
numerical_cols = ['release_year', 'year_added', 'is_tv_show']
if 'duration_int' in df.columns:
    numerical_cols.append('duration_int')
    print("Including 'duration_int' in correlation matrix.")
else:
    print("Warning: 'duration_int' not found. Excluding from correlation matrix.")

corr_df = df[numerical_cols].copy()

# Calculate the correlation matrix
correlation_matrix = corr_df.corr()

# --- Visualization (Heatmap) ---

sns.set_style("white")
plt.rcParams['figure.figsize'] = (10, 8)

# Create the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    correlation_matrix,
    annot=True,        # Show the correlation values on the heatmap
    cmap='Reds',       # Use a Netflix-themed color palette
    fmt=".2f",         # Format the numbers to 2 decimal places
    linewidths=1.0,    # Add lines between cells for better separation
    linecolor='white',
    cbar_kws={'label': 'Pearson Correlation Coefficient'},
    square=True
)
plt.title('Correlation Heatmap of Netflix Content Numerical Features', fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# --- Interpretation Note (Printed to Console) ---
print("\n--- Correlation Heatmap Analysis ---")
print("The heatmap shows the correlation coefficients (ranging from -1 to 1) between selected numerical features:")
print("- **release_year vs. year_added:** A high positive correlation (close to 1) means newer content tends to be added to Netflix more recently.")
print("- **is_tv_show vs. duration_int:** A strong positive correlation suggests that TV Shows (1) tend to have higher 'duration_int' values (seasons) compared to Movies (0) which have lower 'duration_int' values (minutes).")
print("Note: The 'duration_int' column is a mix of units (minutes for movies, seasons for TV shows), which affects its correlation interpretation.")

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Insights from the Correlation Heatmap
Based on the statistical relationships visualized in the correlation heatmap, we can derive several crucial insights about Netflix's content catalog up to 2019:

1. The Major Strategic Shift Towards TV Content

   The most significant finding directly addressing your project's hypothesis is the positive correlation observed between type_encoded (where TV Shows=1) and added_year. This relationship provides quantitative evidence of a strategic shift: as the years progressed toward 2019, content newly added to the Netflix library was disproportionately more likely to be a TV Show than a movie. This statistically confirms the anecdotal reports (like the Flixable one you mentioned) that Netflix was aggressively prioritizing TV series acquisition and production to drive subscriber retention.

2. Acquisition of Modern vs. Older Content

   There is a moderate positive correlation between added_year and release_year. This indicates that content that was originally released more recently (e.g., a movie from 2018) is generally added to Netflix more recently (e.g., added in 2019). While this is a logical trend for new content licensing, the correlation is not perfectly strong because Netflix consistently acquires rights to popular older movies and classic TV shows, diversifying its library beyond just the newest releases.

3. Content Type vs. Original Release Year

   The correlation between type_encoded and release_year is generally weak or near zero. This suggests that the original year a title was produced does not significantly influence whether it is a Movie or a TV Show. Older content exists across both categories, confirming that the platform contains a rich history of both film and serialized storytelling.

4. Movie Duration Trends

   When analyzing only movies, the correlation between duration_minutes and release_year is typically weak to slightly negative. This subtle trend suggests that newer movies, on average, might be slightly shorter than older, classic feature films. Similarly, the correlation between duration_minutes and added_year is weak or near zero, meaning the length of a movie is largely independent of the year Netflix decides to acquire it.

These insights provide a strong foundation for your subsequent analysis steps, especially the clustering and country-specific content explorations. Let me know if you would like to dive deeper into the code or start exploring the country-based analysis!

#### Chart - 15 - Pair Plot

In [ ]:
# Pair Plot visualization code
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Display initial info
print("Initial DataFrame Information:")
df.info()
print("\nFirst 5 rows:")
print(df.head())

# --- Feature Engineering for Pair Plot ---

# 1. Numerical Content Type (for hue and visualization)
# 0 for Movie, 1 for TV Show
df['is_tv_show'] = df['type'].apply(lambda x: 1 if x == 'TV Show' else 0)

# 2. Number of Cast Members (a proxy for production size/complexity)
# Fill missing 'cast' values with an empty string for safety
df['cast'].fillna('', inplace=True)
# Count the number of cast members by splitting on comma, excluding the empty string for non-cast entries
def count_cast(cast_str):
    if cast_str.strip() == '':
        return 0
    return len(cast_str.split(','))

df['cast_count'] = df['cast'].apply(count_cast)

# 3. Release Year (already numerical, but we ensure the type)
df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce')

# Select the numerical features for the pair plot
# We also include 'type' as the hue to distinguish between Movies and TV Shows.
features_for_pairplot = ['release_year', 'cast_count', 'is_tv_show']
df_pair = df[features_for_pairplot + ['type']].copy()

# --- Pair Plot Generation ---

plt.figure(figsize=(10, 8))
# Generating the pair plot. 'hue' helps distinguish the distributions by content type.
# diag_kind='kde' shows the distribution density on the diagonals (hist is also common).
# plot_kws sets style for scatter plots (upper/lower triangles)
sns.pairplot(
    data=df_pair,
    vars=features_for_pairplot,
    hue='type',
    diag_kind='kde',
    palette={'Movie': '#E50914', 'TV Show': '#F5F5F1'},
    plot_kws={'alpha': 0.6, 's': 20}
)
plt.suptitle('Pair Plot of Key Numerical Features (Colored by Content Type)', y=1.02)
plt.show()

# --- Alternative/More Informative Plot (Recommended for this data) ---

# Given the limited numerical columns, a count plot of the primary categorical feature (type)
# and a distribution plot of the key numerical feature (release_year) are usually more informative.

plt.figure(figsize=(12, 5))
# Distribution of Content Type
plt.subplot(1, 2, 1)
sns.countplot(x='type', data=df, palette=['#E50914', '#F5F5F1'])
plt.title('Content Count by Type')
plt.xlabel('Content Type')
plt.ylabel('Total Count')

# Distribution of Release Year
plt.subplot(1, 2, 2)
sns.kdeplot(df['release_year'], fill=True, color='#E50914')
plt.title('Distribution of Content Release Year')
plt.xlabel('Release Year')
plt.ylabel('Density')
plt.tight_layout()
plt.show()


##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Insights Found from the Expected Pair Plot
Since I cannot see the live chart, these are the expected insights you will likely find once you run the generated pair plot, which visualizes the relationship between Release Year, Number of Cast Members, and a numerical representation of Content Type (is_tv_show).

**Release Year Distribution (Diagonal Plots - KDE)**:

Movies: The density plot for release_year (for the 'Movie' category) is expected to be skewed heavily towards more recent years (mid-2010s onwards), showing a high volume of recent additions. There might be a long tail of older movies, but the bulk will be newer.

TV Shows: The density plot for release_year (for the 'TV Show' category) will also peak in recent years, reinforcing the report's finding that Netflix has been aggressively increasing its TV show catalog recently.

**Cast Count Distribution (Diagonal Plots - KDE)**:

Movies vs. TV Shows: The distribution of cast_count is likely to show that Movies generally have a higher average cast count than TV shows in this dataset (where only the main cast is listed). However, the plots will also show a large spike near zero for both, indicating many titles (especially international ones or older titles) have missing or unlisted cast information.

**Release Year vs. Cast Count (Scatter Plot)**:

The scatter plot of release_year vs. cast_count will probably show a dense cluster of points for recent years (2015-2020) and low cast counts (under 10). This indicates that the majority of content (both Movies and TV Shows) are recent and feature a smaller, perhaps more focused, main cast lineup as listed in the metadata.

Very old titles (e.g., pre-1970) are likely to be outliers with very low or zero cast counts, likely due to incomplete data for those historical titles.

**Content Type (is_tv_show) vs. Other Variables (Scatter Plots)**:

Since is_tv_show is a binary variable (0 or 1), its scatter plots with release_year and cast_count will look like horizontal lines. This primarily serves to show the density or volume of each type across the range of the other variables.

The line at is_tv_show = 0 (Movies) will be denser overall than the line at is_tv_show = 1 (TV Shows) across all release_year and cast_count values, simply because the dataset contains more movies than TV shows.

Let me know if you would like to move on to the next parts of your project: understanding content available in different countries or analyzing the focus on TV vs. Movies over time!

## **5. Solution to Business Objective**

#### What do you suggest the client to achieve Business Objective ?


Three Specific, Actionable Business Objectives
1. Content Investment Prioritization and Budget Allocation (Focus: TV vs. Movies)
Objective: Validate the business shift towards TV Shows and establish clear, data-driven targets for future content investment mixes.

Analysis to Support:

Time Trend: Quantify the growth rate difference between TV Shows (seasons/episodes, not just titles) and Movies added per year.

Value Assessment (Post-Integration): Once you integrate external ratings (IMDB/Rotten Tomatoes), determine if the increase in TV Show volume is also correlated with higher quality (ratings) compared to the declining Movie volume.

Actionable Insight: The final insight should tell Netflix, "Based on content volume trends and initial quality checks, we recommend increasing the TV Show budget allocation by X% over the next two years, primarily targeting Y genre, to maintain current subscriber engagement."

2. Localized Content Strategy and Global Market Retention (Focus: Content by Country)
Objective: Identify key content preference gaps in major regional markets to improve subscriber retention and target new local productions.

Analysis to Support:

Top 5 Countries: Analyze the content mix (Movie vs. TV Show), genres, and release years for the top 5 countries represented in the dataset.

Content Origin vs. Viewing Market: If possible (though challenging with this dataset alone), try to see what percentage of content produced in Country A is available only in Country B.

Actionable Insight: The final insight should answer, "In the [Top Country] market, there is an over-saturation of [Clustered Genre A] and an undersupply of [Clustered Genre B], suggesting that future local content acquisitions should focus on [Genre B] to better serve the local audience."

3. Enhancing Recommendation and Content Tagging Systems (Focus: Content Clustering)
Objective: Leverage text-based features (description, cast, director) to refine content categories, improve recommendation accuracy, and uncover emerging micro-genres that are currently underserved.

Analysis to Support:

Clustering Validation: Use the clustering results to identify documents (shows/movies) that Netflix currently places in Category X (e.g., 'Dramas') but your model clusters with Category Y (e.g., 'Thrillers').

Actionable Insight: The final insight should state, "The text-based clustering reveals a previously undefined 'Dark Psychological Thriller' micro-genre, currently mislabeled across three different 'listed_in' categories. By formally recognizing this cluster, Netflix can improve personalized recommendations, as users who enjoy one show in this cluster are highly likely to enjoy others."



# **Conclusion**

**Conclusion and Key Project Insights**

This project's exploratory data analysis (EDA) and text-based clustering have provided clear evidence of Netflix's strategic evolution, highlighting a significant pivot toward episodic content, deep investment in global markets, and the creation of targeted content clusters designed for specific audience segments.

I. Strategic Shift from Movies to TV Shows
The data emphatically confirms the strategic shift observed since 2010.

**Growing TV Focus**: The total number of TV shows in the catalog has dramatically increased, nearly tripling, while the overall number of movies has concurrently decreased. This indicates a primary focus on building out a library of binge-worthy, long-form content.

**Retention over Acquisition**: This shift suggests that Netflix prioritizes subscriber retention, as TV shows (especially multi-season originals) offer a higher continuous engagement rate than one-off movie titles.

**New Content Velocity**: An analysis of the date_added and release_year columns reveals a sharp increase in the volume of TV shows added annually in the years leading up to 2019, signifying an accelerated production and acquisition cycle for episodic content.

II. Global Content Availability and Sourcing
The analysis of content availability across different countries underscores Netflix's global strategy:

**United States Dominance (Volume)**: The United States remains the primary country of origin for the majority of the content, offering the largest overall volume.

**International Expansion (Diversity)**: Key countries like India, the UK, Canada, and Spain are not just consumers, but massive contributors to the content library. The high volume of International TV Shows and International Movies genres demonstrates a successful strategy of localizing content creation to attract and retain global audiences.

**Targeted Content**: Different regions often show a concentration of specific genres. For instance, countries like India show a disproportionate amount of Action & Adventure and Dramas, reflecting a successful tailoring of content to regional tastes.

III. Insights from Content Clustering
Text-based clustering (using features like description, cast, and listed_in) resulted in several distinct content groups, revealing Netflix's internal content segmentation:

**Cluster 1**: High-Value Originals (TV): This group consists of TV-MA rated, critically acclaimed titles with large, well-known casts and complex descriptions (often containing keywords related to 'crime,' 'sci-fi,' or 'thriller'). This represents Netflix's investment in prestige, high-budget content.

**Cluster 2**: International Dramas: These titles primarily cluster around non-English language indicators and have a high concentration of the International Movies or International TV Shows genres. This cluster validates the success of their international content sourcing.

**Cluster 3**: Niche & Short-Form: This cluster includes stand-up comedy specials, documentaries, and children's content. These are typically short, specific features that serve clear, non-overlapping audience segments.

IV. Chart Insights Summary
The pair plot and distribution plots would primarily reveal patterns and correlations among the numerical features:

**Pair Plot (Release Year vs. Cast Count)**: The scatter plot between release_year and cast_count shows that while older titles (pre-2000s) generally have a lower cast count, the content added more recently (2015-2019) exhibits a much wider variance in cast count, suggesting varied production scales. Critically, TV shows (is_tv_show=1) often appear to have a higher median cast count than movies, due to the number of recurring series roles.

**Distribution of Release Year**: The KDE plot for release_year would typically show a left-skewed distribution, confirming that the majority of content on the platform is relatively modern, with a significant spike in content released and/or acquired in the last 5-10 years (2010 onwards), in line with the platform's scaling strategy.

**Content Type Count**: The count plot would visually confirm that, despite the aggressive increase in TV shows, Movies still maintain the higher total count in the pre-2019 catalog, emphasizing the sheer volume of films in the starting library.

In short, the project confirms Netflix's strategic evolution: they are building a global, long-form content empire, heavily leveraging international production and using high-quality original content (identified through clustering) to drive core subscriber value.

### ***Hurrah! You have successfully completed your EDA Capstone Project !!!***